<a href="https://colab.research.google.com/github/anand-therattil/NaturalLanguageProcessing/blob/main/SpamClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 

# To be used for the drive connection
from google.colab import drive
drive.mount(r'/content/drive')

import re 
import nltk
from nltk import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# nltk.download('all')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
messages = pd.read_csv(r'drive/MyDrive/Colab Notebooks/Dataset/SMSSpamCollection',sep="\t",names=['label','messages'])

In [3]:
messages.head()

,label,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
def preProcessing(sentence):
  sentence=re.sub('[^a-zA-Z]',' ',sentence)
  sentence= sentence.lower()
  return sentence

#Stemming
def stemming(sentences):
  stemmer= PorterStemmer()
  sentences= preProcessing(sentences)
  words= nltk.word_tokenize(sentences)
  features =[stemmer.stem(word) for word in words if word not in set(stopwords.words('english'))]
  features = " ".join(features)
  return features
# Lemmetization
def lemmatization(sentences):
  lemmatizer = WordNetLemmatizer()
  sentences= preProcessing(sentences)
  words=nltk.word_tokenize(sentences)
  features =[lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
  features = " ".join(features)
  return features
     

In [33]:
corpus =[]
for sentence in messages['messages']:
  corpus.append(lemmatization(sentence))


In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cv =CountVectorizer(max_features=1000)
X= cv.fit_transform(corpus).toarray()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf =TfidfVectorizer()
X= tf.fit_transform(corpus).toarray()

In [35]:
y=pd.get_dummies(messages['label'])
y= y.iloc[:,1].values

In [36]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)

In [37]:
from sklearn.naive_bayes import MultinomialNB
spam_detect= MultinomialNB().fit(x_train,y_train)


In [38]:
y_predict =spam_detect.predict(x_test)
print(y_predict)

[0 0 1 ... 0 0 1]


In [39]:
from sklearn.metrics import accuracy_score
testing_acc = accuracy_score(y_predict,y_test)
print(testing_acc*100)

95.87443946188341
